### 데이터 준비

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from scipy.io import loadmat

# 데이터 불러오기 및 처리 함수 정의
def load_data(battery):
    mat = loadmat(f'C:/LEE/batterydata/{battery}.mat')
    dataset = []
    capacity_data = []
    counter = 0

    for i in range(len(mat[battery][0, 0]['cycle'][0])):
        row = mat[battery][0, 0]['cycle'][0, i]
        if row['type'][0] == 'discharge':
            capacity = row['data'][0][0]['Capacity'][0][0]
            for j in range(len(row['data'][0][0]['Voltage_measured'][0])):
                temperature_measured = row['data'][0][0]['Temperature_measured'][0][j]
                current_measured = row['data'][0][0]['Current_measured'][0][j]
                voltage_measured = row['data'][0][0]['Voltage_measured'][0][j]
                dataset.append([counter + 1, temperature_measured, current_measured, voltage_measured, capacity])
            capacity_data.append([counter + 1, capacity])
            counter += 1

    return pd.DataFrame(dataset, columns=['cycle', 'temperature_measured', 'current_measured', 'voltage_measured', 'capacity']), \
           pd.DataFrame(capacity_data, columns=['cycle', 'capacity'])

# 데이터 불러오기
data_B0005, capacity_B0005 = load_data('B0005')
data_B0006, capacity_B0006 = load_data('B0006')
data_B0007, capacity_B0007 = load_data('B0007')
data_B0018, capacity_B0018 = load_data('B0018')

# 공칭 용량 정의 및 SOH 계산
initial_capacity = capacity_B0005['capacity'].iloc[0]  # B0005의 초기 용량을 공칭 용량으로 사용
for capacity_data in [capacity_B0005, capacity_B0006, capacity_B0007, capacity_B0018]:
    capacity_data['SOH'] = capacity_data['capacity'] / initial_capacity

# Train data: B0005, B0006, B0018 | Test data: B0007
train_data = pd.concat([data_B0005, data_B0006, data_B0018], ignore_index=True)
test_data = data_B0007

### 스케일링 (온도,전류,전압 만)

In [2]:
# Feature scaling (온도, 전류, 전압만 사용)
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data[['temperature_measured', 'current_measured', 'voltage_measured']])
test_scaled = scaler.transform(test_data[['temperature_measured', 'current_measured', 'voltage_measured']])

# Sequence preparation for LSTM
def create_sequences(data, sequence_length=20):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        x = data[i:i+sequence_length, :]  # Features: 온도, 전류, 전압
        y = train_data['capacity'].iloc[i+sequence_length] / initial_capacity  # Capacity to SOH
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

sequence_length = 20
X_train, y_train = create_sequences(train_scaled, sequence_length)
X_test, y_test = create_sequences(test_scaled, sequence_length)

In [3]:
import joblib

joblib.dump(scaler, 'scaler_soh.pkl')

['scaler_soh.pkl']

### LSTM 모델 학습

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# LSTM 모델 정의
model = Sequential([
    LSTM(100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    Dense(1)  # Output layer for SOH prediction
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
history = model.fit(X_train, y_train, epochs=200, batch_size=40, validation_split=0.2, verbose=1)

Epoch 1/200


c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2709/2709 ━━━━━━━━━━━━━━━━━━━━ 26s 9ms/step - loss: 0.0337 - val_loss: 0.0044
Epoch 2/200
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 0.0098 - val_loss: 0.0040
Epoch 3/200
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - loss: 0.0073 - val_loss: 0.0051
Epoch 4/200
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 26s 10ms/step - loss: 0.0066 - val_loss: 0.0044
Epoch 5/200
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 27s 10ms/step - loss: 0.0062 - val_loss: 0.0071
Epoch 6/200
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - loss: 0.0056 - val_loss: 0.0041
Epoch 7/200
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - loss: 0.0044 - val_loss: 0.0036
Epoch 8/200
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 26s 9ms/step - loss: 0.0038 - val_loss: 0.0051
Epoch 9/200
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 26s 9ms/step - loss: 0.0037 - val_loss: 0.0046
Epoch 10/200
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 25s 9ms/step - loss: 0.0037 - val_loss: 0.0044
Epoch 11/200
2709/2709 ━━━━━━━━━━━━━━━━━━━━ 26s 9ms/step - loss: 0.0036 - val_loss: 0.0033
Epoch 12/200
2709/

In [5]:
model.save('lstm_b000(5,6,7,18).keras')

In [4]:
from tensorflow.keras.models import load_model

# 모델 파일 경로
model_path = 'lstm_b000(5,6,7,18).keras'

# 모델 불러오기
model_soh = load_model(model_path)

# 모델 요약 출력 (옵션)
model_soh.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        10,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,555 (127.17 KB)

 Trainable params: 10,851 (42.39 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 21,704 (84.79 KB)

### 예측 & 평가

In [6]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict SOH on test set
y_pred = model_soh.predict(X_test)

# Calculate RMSE and R^2 for evaluation
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Test RMSE for SOH: {rmse}")
print(f"Test R^2 Score for SOH: {r2}")

1571/1571 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step
Test RMSE for SOH: 0.13911533467674128
Test R^2 Score for SOH: -1.0066590600259508


### 실시간 예측?


In [11]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import joblib
import numpy as np

scaler = joblib.load('scaler_soh.pkl')

def predict_soh_real_time(model, scaler, sequence):
    # Scale sequence
    sequence_scaled = scaler.transform(sequence)
    sequence_scaled = np.expand_dims(sequence_scaled, axis=0)  # Reshape for LSTM
    
    # Predict SOH
    predicted_soh = model.predict(sequence_scaled)[0][0]
    return predicted_soh

# Example usage: Provide a sequence of real-time data
real_time_sequence = [
    [23.8, 0.5, 0.02],  # 가장 오래된 측정값[온도,전류,전압]
    [25.4, 0.5, 0.02],
    [25.4, 0.5, 0.02],
    [25.5, 0.6, 0.02],
    [25.3, 0.4, 0.02],
    [25.2, 0.5, 0.02],
    [25.3, 0.5, 0.02],
    [25.3, 0.5, 0.02],
    [25.2, 0.4, 0.02],
    [25.1, 0.6, 0.02],
    [25.2, 0.5, 0.02],
    [25.3, 0.5, 0.02],
    [25.4, 0.4, 0.02],
    [25.3, 0.5, 0.02],
    [25.2, 0.5, 0.02],
    [25.4, 0.6, 0.02],
    [25.4, 0.5, 0.02],
    [25.3, 0.5, 0.02]  # 가장 최신 측정값
]

predicted_soh = predict_soh_real_time(model_soh, scaler, real_time_sequence)
print(f"Predicted SOH in real-time: {predicted_soh}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Predicted SOH in real-time: 327.7948303222656


c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
